In [ ]:
# default_exp core
%load_ext lab_black

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import os

if os.name == "nt":
    git_exe = r"C:\Program Files\Git\cmd\git.exe"
else:
    git_exe = "/usr/bin/git"
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = git_exe
import sys
import shlex
import argparse
import pandas as pd
from git import Repo
from pathlib import Path
from collections import namedtuple
from subprocess import Popen, PIPE


python = sys.executable
home = Path(os.path.expanduser("~"))
blogslicer = Path(home / Path("github/blogslicer/blogslicer/core.py"))
if hasattr(__builtins__, "__IPYTHON__") or __name__ != "__main__":
    from IPython.display import display, Markdown

    h1 = lambda text: display(Markdown(f"# {text}"))
    h2 = lambda text: display(Markdown(f"## {text}"))
    h3 = lambda text: display(Markdown(f"### {text}"))

    file = "sites.csv"
else:
    h1 = lambda text: print(f"# {text}")
    h2 = lambda text: print(f"## {text}")
    h3 = lambda text: print(f"## {text}")

    aparser = argparse.ArgumentParser()
    add_arg = aparser.add_argument
    add_arg("-f", "--file", required=True)
    args = aparser.parse_args()
    file = args.file

h1("Generaring sites...")
file_obj = Path(file)
df = pd.read_csv(file_obj, delimiter="|")
df = df.applymap(lambda x: x.strip())
df.columns = [x.strip() for x in df.columns]
Site = namedtuple("Site", "path, apex, title, gaid, tagline")


def git(args):
    cmd = [git_exe] + shlex.split(args)
    process = Popen(
        args=cmd,
        cwd=here,
        stdout=PIPE,
        stderr=PIPE,
        shell=False,
        bufsize=1,
        universal_newlines=True,
    )
    print(f"GIT: {shlex.join(cmd)}")
    for line in process.stdout:
        print(line)
        sys.stdout.flush()


h2(f"Python: {python}")
h2(f"Blogslicer: {blogslicer}")
print()
for index, series in df.iterrows():
    site = Site(**series.to_dict())
    h3(site.apex)
    here = Path(home / site.path)

    # Blog Slicer
    cmd = f'{python} {blogslicer} -p {here} -t "{site.title}" -s "blog" -a "Mike Levin"'
    print(cmd, end="\n\n")
    with Popen(args=cmd, cwd=here, stdout=PIPE, stderr=PIPE, shell=True) as pout:
        for line in pout.stdout.readlines():
            print(line.decode().strip())

    git("add _posts/*")
    git('commit -am "Publising Blog Posts"')
    git("push")


h2("Done!")

# Generaring sites...

## Python: C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe

## Blogslicer: C:\Users\mikle\github\blogslicer\blogslicer\core.py

### GuerillaTech.com

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\GuerillaTech.com -t "Guerilla Tech" -s "blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\GuerillaTech.com/_posts/2022-04-22-post-1.md
GIT: 'C:\Program Files\Git\cmd\git.exe' add '_posts/*'
GIT: 'C:\Program Files\Git\cmd\git.exe' commit -am 'Publising Blog Posts'
On branch main

Your branch is up to date with 'origin/main'.



nothing to commit, working tree clean

GIT: 'C:\Program Files\Git\cmd\git.exe' push


...

### WeeklyReports.zd

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\WeeklyReports.zd -t "Mike Levin's Weekly Reports" -s "blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\WeeklyReports.zd/_posts/2022-05-14-post-2.md
## FILE: C:\Users\mikle\github\WeeklyReports.zd/_posts/2022-05-07-post-1.md
GIT: 'C:\Program Files\Git\cmd\git.exe' add '_posts/*'
GIT: 'C:\Program Files\Git\cmd\git.exe' commit -am 'Publising Blog Posts'
On branch main

Your branch is up to date with 'origin/main'.



nothing to commit, working tree clean

GIT: 'C:\Program Files\Git\cmd\git.exe' push


### WhatsaMetaFor.io

C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\WhatsaMetaFor.io -t "What's A Meta For" -s "blog" -a "Mike Levin"

## FILE: C:\Users\mikle\github\WhatsaMetaFor.io/_posts/2022-05-06-post-4.md
## FILE: C:\Users\mikle\github\WhatsaMetaFor.io/_posts/2022-05-03-post-3.md
## FILE: C:\Users\mikle\github\WhatsaMetaFor.io/_posts/2022-04-30-post-2.md
## FILE: C:\Users\mikle\github\WhatsaMetaFor.io/_posts/None-post-1.md
GIT: 'C:\Program Files\Git\cmd\git.exe' add '_posts/*'
GIT: 'C:\Program Files\Git\cmd\git.exe' commit -am 'Publising Blog Posts'
On branch main

Your branch is up to date with 'origin/main'.



nothing to commit, working tree clean

GIT: 'C:\Program Files\Git\cmd\git.exe' push


## Done!

In [ ]:
!nbdev_clean_nbs
!nbdev_build_lib